In [ ]:
import gensim
import numpy as np
import string
import tensorflow as tf
import Placeholders
import datetime
import random as random

In [ ]:
with tf.Session() as sess:
    x = tf.add(3,4).eval()
    print(x)

In [ ]:
x = Placeholders.sentence
y_ = Placeholders.class_labels
c = Placeholders.class_centroids

In [ ]:
def get_cluster_based_labels(fc_layer_representation):
    label_to_cluster_map = lambda x:  c[x]
    indexed_centroids = tf.map_fn(label_to_cluster_map, y_)
    x_distances = np.exp(-(numpy.linalg.norm(x - indexed_centroids)))
    sum_x_distances = np.log(np.sum(x_distances))
    logits = np.divide(x_distances, sum_x_distances)
    return logits
    

In [ ]:
def one_hot(vec, vals=Placeholders.n_classes):
    n = vec.shape[1]
    print ("len",n)
    out = np.zeros((n, vals))
    out[np.arange(n), vec] = 1
    return out

In [ ]:
def train(sess):
    hidden1 =tf.layers.dense(Placeholders.sentence, Placeholders.hidden1_neurons)
    hidden2 =tf.layers.dense(Placeholders.sentence, Placeholders.hidden2_neurons)
    cluster_based_labels = hidden2#get_cluster_based_labels(hidden2)
    logits = tf.layers.dense(cluster_based_labels, Placeholders.n_classes)
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits= logits,
                                                                   labels=Placeholders.class_labels))
    loss = tf.reduce_mean(cross_entropy)
    train_step = tf.train.AdamOptimizer(1e-5).minimize(loss)

    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Placeholders.class_labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    STEPS = 3000
    MINIBATCH_SIZE = 20
    
    print ("Starting at:" , datetime.datetime.now())
    sess.run(tf.global_variables_initializer())
    print ("Initialization done at:" , datetime.datetime.now())
    
    n_samples = 100
    mu, sigma = 10, 10
    X = np.array([np.random.normal(mu, sigma, Placeholders.word_vec_dim) for i in range(n_samples)])
    Y = np.zeros(n_samples, dtype=np.int16)
    mu, sigma = 50, 10
    X = np.vstack((X, np.array([np.random.normal(mu, sigma, Placeholders.word_vec_dim) for i in range(n_samples)])))
    Y = np.vstack((Y,np.ones(n_samples,dtype=np.int16))).reshape(-1,n_samples *2 )
    print(Y)
    print (X.shape,Y.shape)
    Y = one_hot(Y)
    print (X.shape,Y.shape)
    C = np.ones(Placeholders.word_vec_dim)*10
    C = np.vstack((C,np.ones(Placeholders.word_vec_dim)*50))
    
    #sess.run(train_step, feed_dict={x: X, y_: Y, c:C})
    
    for i in range(STEPS):
        random_index = random.randint(0,n_samples * 2 - MINIBATCH_SIZE - 1)
        random_X = X[random_index + MINIBATCH_SIZE,:].reshape(-1, Placeholders.word_vec_dim)
        random_Y = Y[random_index + MINIBATCH_SIZE,:].reshape(-1, Placeholders.n_classes)
        print(random_X.shape)
        sess.run(train_step, feed_dict={x: random_X, y_: random_Y, c:C})
        acc = sess.run(accuracy, feed_dict={x: X, y_: Y, c:C})*100
        print (acc)

    

In [ ]:
with tf.Session() as sess:
    train(sess)